# Setup

In [ ]:
!pip install groq
!pip install diff-match-patch
!pip install python-Levenshtein
import pandas as pd
import json
from google.colab import userdata
from groq import Groq
import Levenshtein
import difflib
import pandas as pd
import json
import difflib
import time
import re
import re
import unicodedata
from difflib import SequenceMatcher
api_key = userdata.get("GROQ_API_KEY")
client = Groq(api_key=api_key)



def get_response_and_time(prompt):
  start_time = time.time()
  response = client.chat.completions.create(
  model="llama-3.1-70b-versatile",
  messages=[
    {"role": "user", "content": prompt }
    ]
  )
  end_time = time.time()
  return response.choices[0].message.content, end_time - start_time

response, duration = get_response_and_time("Say 'hello world' and nothing else")
print(response)
print(duration)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.3/106.3 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.4/177.4 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 43.7 MB/s eta 0:00:00
hello world
0.11774945259094238


In [ ]:
df2 = pd.read_excel('/content/grupo 1 (revisado).xlsx')
marked = " ".join(df2.iloc[:, 0].astype(str).tolist())
print(marked)

Vamos lá, <hes é/>  boa tarde todo mundo. Bem-vindos a este debate. Promovido pelo grupo de pesquisa do lacina debate em educação. Eu me chamo David e gostaria de apresentar aqui, quem está comigo, Klaywert que também faz parte do grupo. Mas aí a gente tem também Bryan e Hellen, que são tirando Hellen, a maioria estudante do mestrado, Hellen é estudante da graduação. <hes é/>, e esse projeto está sobre orientação do professor Campelo, que é o professor <erro que vocês conhecem/> <corr que vocês conheceram> agorinha. que vocês viram agora certo, okay, a razão pela qual nos reunimos aqui hoje, <rep é é/>  gerar uma base de dados de voz que posteriormente será transcrita e utilizada para avaliação e treinamento de modelos de inteligência artificial. Nosso foco está entre vários tópicos relacionados a debates, fazendo dessas iniciativa não apenas oportunidade de aprendizado e crescimento mas também uma contribuição valiosa para avanço de IA. 
Vocês assinaram o termo, né? Todo mundo já assi

# Código que limpa o texto marcado de disfluências e armazena elas em um array

In [ ]:
def produz_array(text):
    tags_pattern = r'<(rep|corr|erro|hes) ([^<>]*)/?>'

    disfluencias = []  # Para armazenar todas as disfluências

    def replace_tags(match):

        tag = match.group(1)
        content = match.group(2).strip()

        # Remove '/', '/>', '/>>', '//>' do final do conteúdo
        content = re.sub(r'[/]+>?$', '', content).strip()

        # Adiciona a disfluência à lista
        if tag in ['rep', 'hes', 'erro', 'corr']:
            disfluencias.append((tag, content))

        return content

    # Aplicando a substituição de todas as tags no texto
    texto_disfluente = re.sub(tags_pattern, replace_tags, text)

    return texto_disfluente, disfluencias

texto_disfluente, disfluencias = produz_array(marked)

print("Texto Disfluente:\n", texto_disfluente)
print("\nDisfluências Encontradas:\n", disfluencias)


Texto Disfluente:
 Vamos lá, é  boa tarde todo mundo. Bem-vindos a este debate. Promovido pelo grupo de pesquisa do lacina debate em educação. Eu me chamo David e gostaria de apresentar aqui, quem está comigo, Klaywert que também faz parte do grupo. Mas aí a gente tem também Bryan e Hellen, que são tirando Hellen, a maioria estudante do mestrado, Hellen é estudante da graduação. é, e esse projeto está sobre orientação do professor Campelo, que é o professor que vocês conhecem que vocês conheceram agorinha. que vocês viram agora certo, okay, a razão pela qual nos reunimos aqui hoje, é é  gerar uma base de dados de voz que posteriormente será transcrita e utilizada para avaliação e treinamento de modelos de inteligência artificial. Nosso foco está entre vários tópicos relacionados a debates, fazendo dessas iniciativa não apenas oportunidade de aprendizado e crescimento mas também uma contribuição valiosa para avanço de IA. 
Vocês assinaram o termo, né? Todo mundo já assinou o termo, entã

# Faz uma limpeza para que o texto corrigido humanamente sem anotações de disfluência fique 100% livre de ruídos

In [ ]:


def limpar_texto(texto):
    # Remover todas as tags <hes .../> e <erro .../> junto com seus conteúdos
    texto = re.sub(r'<hes.*?/>', '', texto)
    texto = re.sub(r'<erro.*?/>', '', texto)

    # Manter o conteúdo dentro de <corr .../> e remover a tag
    texto = re.sub(r'<corr (.*?)\/>', r'\1', texto)

    def remove_repeticoes(match):
        # Captura o conteúdo da tag <rep> e converte para minúsculas
        conteudo = match.group(1).strip().lower()
        # Divide o conteúdo em uma lista de palavras

        # Remove pontuação do conteúdo
        conteudo = re.sub(r'[^\w\s]', '', conteudo)
        palavras = conteudo.split()

        # String para armazenar as palavras únicas
        ocorrencia_unica = ""
        # Set para armazenar palavras únicas
        palavras_armazenadas = set()

        # Percorre cada palavra do conteúdo original
        for palavra in palavras:
            # Adiciona a palavra se ela ainda não estiver em palavras_armazenadas
            if palavra not in palavras_armazenadas:
                palavras_armazenadas.add(palavra)
                ocorrencia_unica += palavra + " "

        # Remove o espaço extra no final e retorna o resultado
        return ocorrencia_unica.strip()


    texto = re.sub(r'<rep (.*?)\/>', remove_repeticoes, texto)

    # Remover espaços extras gerados após as substituições
    texto = re.sub(r'\s+', ' ', texto).strip()

    return texto

texto_limpo = limpar_texto(marked)
print(texto_limpo)


Vamos lá, boa tarde todo mundo. Bem-vindos a este debate. Promovido pelo grupo de pesquisa do lacina debate em educação. Eu me chamo David e gostaria de apresentar aqui, quem está comigo, Klaywert que também faz parte do grupo. Mas aí a gente tem também Bryan e Hellen, que são tirando Hellen, a maioria estudante do mestrado, Hellen é estudante da graduação. , e esse projeto está sobre orientação do professor Campelo, que é o professor que vocês conheceram> agorinha. que vocês viram agora certo, okay, a razão pela qual nos reunimos aqui hoje, <rep é é gerar uma base de dados de voz que posteriormente será transcrita e utilizada para avaliação e treinamento de modelos de inteligência artificial. Nosso foco está entre vários tópicos relacionados a debates, fazendo dessas iniciativa não apenas oportunidade de aprendizado e crescimento mas também uma contribuição valiosa para avanço de IA. Vocês assinaram o termo, né? Todo mundo já assinou o termo, então a gente pode pular essa parte cada d

# Contador de Disfluências Removidas

In [ ]:
def contar_ocorrencias_texto(texto, dicionario):
    # Inicializando o dicionário de contagem de ocorrências
    ocorrencias_llm = {key: 0 for key in dicionario}

    # Adiciona espaços ao redor do texto para capturar padrões no início e fim
    texto = f" {texto} "

    # Pontuações consideradas após o padrão
    pontuacoes = r'[,.!?;:]?'

    # Iterando sobre as chaves do dicionário
    for chave in dicionario:
        # Adiciona espaços ao redor da chave e permite pontuações depois da chave
        padrao = rf' {re.escape(chave)}{pontuacoes} '
        # Usando expressão regular para encontrar todas as ocorrências da chave no texto
        ocorrencias = re.findall(padrao, texto)
        # Atualizando a contagem no dicionário 'ocorrencias_llm'
        ocorrencias_llm[chave] = len(ocorrencias)

    return ocorrencias_llm

def subtrair_dicionarios(dict1, dict2):
    # Inicializando o dicionário de resultado para a diferença
    resultado = {}

    # Conjunto de todas as chaves presentes em ambos os dicionários
    todas_chaves = set(dict1.keys()).union(set(dict2.keys()))

    # Calculando a diferença para cada chave
    for chave in todas_chaves:
        # Contagem em dict1 (dicionário original)
        contagem1 = dict1.get(chave, 0)
        # Contagem em dict2 (dicionário resultante)
        contagem2 = dict2.get(chave, 0)
        # Calculando a diferença
        diferenca = contagem1 - contagem2
        # Adicionando ao dicionário de resultado apenas se a diferença for não zero
        if diferenca != 0:
            resultado[chave] = diferenca

    return resultado

def preenche_dic_disfluentes(disfluencias):
  # Inicializando os dicionários de hesitações, repetições, erros e correções
    hes = {}
    rep = {}
    erro = {}
    corr = {}

    # Preenchendo os dicionários 'hes', 'rep', 'erro' e 'corr'
    for tag, content in disfluencias:
        if tag == 'hes':
            hes[content] = hes.get(content, 0) + 1
        elif tag == 'rep':
            rep[content] = rep.get(content, 0) + 1
        elif tag == 'erro':
            erro[content] = erro.get(content, 0) + 1
        elif tag == 'corr':
            corr[content] = corr.get(content, 0) + 1

    return hes, rep, erro, corr
hes, rep, erro, corr = preenche_dic_disfluentes(disfluencias)

def count_disfluencias_in_clean(clean_text, hes, rep, erro, corr):
        # Contando ocorrências no texto
    hes_llm = contar_ocorrencias_texto(clean_text, hes)
    rep_llm = contar_ocorrencias_texto(clean_text, rep)
    erro_llm = contar_ocorrencias_texto(clean_text, erro)
    corr_llm = contar_ocorrencias_texto(clean_text, corr)


    return hes_llm, rep_llm, erro_llm, corr_llm



# Código para fazer o balanço da limpeza da LLM

In [ ]:
def count_matches(manual_dict, llm_dict):
    """
    Counts how many times the LLM correction matches the manual correction.

    Args:
    - manual_dict (dict): The dictionary with the manual correction.
    - llm_dict (dict): The dictionary with the LLM correction.

    Returns:
    - int: The count of keys with matching values.
    """
    # Initialize the counter for matches
    matches = 0
    length_of_incorrect_matches = 0

    # Iterate through all keys in the manual correction dictionary
    for key in manual_dict:
        # Check if the key exists in the LLM correction dictionary and has the same value
        if key in llm_dict and manual_dict[key] == llm_dict[key]:
            matches += 1  # Increment the counter if the values match
        else:
          errors = abs(manual_dict[key] - llm_dict[key])
          length_of_incorrect_matches += abs(len(key)) * errors
    return matches, length_of_incorrect_matches

def calcular_metricas(acertos, total_chaves):
    """
    Calcula as métricas de comparação entre acertos da LLM e o total de chaves.

    Args:
    - acertos (int): Número de acertos da LLM.
    - total_chaves (int): Número total de chaves no dicionário manual.

    Returns:
    - porcentagem_acertos (float): Percentual de acertos da LLM.
    - erros (int): Número de chaves que a LLM não identificou corretamente.
    """
    porcentagem_acertos = (acertos / total_chaves) * 100 if total_chaves > 0 else 0
    erros = total_chaves - acertos
    return porcentagem_acertos, erros

def exibir_metricas(resultados):
    """
    Exibe as métricas de comparação entre a LLM e o manual para cada categoria.

    Args:
    - resultados (dict): Dicionário com as categorias e tuplas de (acertos, total_chaves).
    """
    total_acertos = 0
    total_chaves = 0
    categorias_erro = ['Repetições', 'Hesitações', 'Erros']

    # Calcula e exibe as métricas para cada categoria
    for categoria, (acertos, total) in resultados.items():
        porcentagem_acertos, erros = calcular_metricas(acertos, total)
        print(f'{categoria}:')
        print(f'  Acertos da LLM: {acertos}')
        print(f'  Total de chaves no manual: {total}')
        print(f'  Porcentagem de acertos: {porcentagem_acertos:.2f}%')
        print(f'  Número de chaves não corretamente identificadas: {erros}')
        print()

        # Somando para o cálculo da média de acertos
        if categoria in categorias_erro:
            total_acertos += acertos
            total_chaves += total

    # Calcula a porcentagem média de acertos para as categorias de erro
    media_acertos = (total_acertos / total_chaves) * 100 if total_chaves > 0 else 0
    print(f'Média de porcentagem de acertos entre Hesitações, Erros e Repetições: {media_acertos:.2f}%')


# Código para mensurar as alterações e integridade

In [ ]:
def calculate_metrics(original, processed):
    # Calcula a distância de Levenshtein
    levenshtein_distance = Levenshtein.distance(original, processed)

    # Calcula a similaridade de Levenshtein
    levenshtein_similarity = Levenshtein.ratio(original, processed)

    # Calcula a distância de edição
    edit_distance = Levenshtein.distance(original, processed)

    original_length = len(original)
    processed_length = len(processed)

    original_word_count = len(original.split())
    processed_word_count = len(processed.split())

    return {
        'Levenshtein Distance': f'{levenshtein_distance:,}',
        'Levenshtein Similarity': f'{levenshtein_similarity * 100:.2f}%',
        'Edit Distance': f'{edit_distance:,}',
        'Original Length': f'{original_length:,}',
        'Processed Length': f'{processed_length:,}',
        'Original Word Count': f'{original_word_count:,}',
        'Processed Word Count': f'{processed_word_count:,}'
    }


def adjust_levenshtein_distance(data, not_to_be_considered):
    """
    Adjusts the Levenshtein Distance by excluding a specified number of edits
    and recalculates the Levenshtein Similarity.

    Parameters:
    data (dict): A dictionary containing Levenshtein metrics:
                 {'Levenshtein Distance': str, 'Processed Length': str}
    not_to_be_considered (int): The number of edits that should not be considered.

    Returns:
    dict: A dictionary with the adjusted Levenshtein Distance and Similarity.
    """
    # Extract values from the dictionary and convert them to integers
    levenshtein_distance = int(data['Levenshtein Distance'].replace(',', ''))
    processed_length = int(data['Processed Length'].replace(',', ''))

    # Adjust the Levenshtein Distance
    adjusted_distance = levenshtein_distance - not_to_be_considered

    # Recalculate the Levenshtein Similarity
    adjusted_similarity = (1 - (adjusted_distance / processed_length)) * 100

    # Return the adjusted values as a dictionary
    return {
        'Adjusted Levenshtein Distance': adjusted_distance,
        'Adjusted Levenshtein Similarity': f'{adjusted_similarity:.2f}%'
    }


# Prompt 1.1 (Zero-shot no context)

In [ ]:
prompt = f"""
Please remove disfluencies from the following Portuguese text transcription without changing the original text.
Ensure that aside from the removal of this disfluencies, no other changes are made to the text. You should NOT change text that does not refer to disfluencies. Return only the cleaned text with no additional information. Remember the text must be in its original full size, but without the disfluencies in it.


Text: {texto_disfluente}
"""
response1, duration1 = get_response_and_time(prompt)

print(response1)
print('A limpeza foi feita em ', duration1)

Vamos lá, é boa tarde a todos. Bem-vindos a este debate. Promovido pelo grupo de pesquisa do Lacina, debate em educação. Eu me chamo David e gostaria de apresentar aqui quem está comigo, Klaywert, que também faz parte do grupo. Mas aí a gente tem também Bryan e Hellen, que são, tirando Hellen, a maioria estudantes do mestrado. Hellen é estudante da graduação. E esse projeto está sobre a orientação do professor Campelo, que é o professor que vocês conhecem.

A razão pela qual nos reunimos aqui hoje é gerar uma base de dados de voz que posteriormente será transcrita e utilizada para avaliação e treinamento de modelos de inteligência artificial. Nosso foco está entre vários tópicos relacionados a debates, fazendo dessas iniciativas não apenas oportunidades de aprendizado e crescimento mas também uma contribuição valiosa para o avanço de IA.

Vocês assinaram o termo, certo? Todo mundo já assinou o termo, então a gente pode pular essa parte. Cada debatedor vai receber uma identificação. Voc

## prompt

In [ ]:
prompt_1 = """Gravação iniciada. Minha identificação é o número aluno um, estou contribuindo para a pesquisa no Brasil. Minha identificação é aluno 2, estou contribuindo para a pesquisa no Brasil. Minha identificação é a número 3. Estou contribuindo com a pesquisa no Brasil. Minha eficação é o número 4. Estou contribuindo com a pesquisa do Brasil. Minha identificação número 5, estou contribuindo com a pesquisa no Brasil. Excelente. Pois bem, agora eu explicarei as regras deste debate, e então, antes de mergulharmos em nossa discussão, é essencial. Todos compreendam e sigam as reuniões de debate. Essas regras foram ampliadas para garantir um debate justo e ordenado para todos os envolvidos. A única principal regra é: sem interrupções. Os debatedores não devem interromper uns aos outros enquanto alguém estiver falando. Se você deseja contribuir para alguma discussão ou oferecer um contra argumento, por favor, levante a mão e aguarde um moderador, no caso eu lhe conceder a palavra, uma vez que tenha sido autorizada a palavra, você terá a palavra e poderá expressar seus pensamentos ou responder aos outros. vamos seguir um formato estruturado dividido em 3 momentos distintos, cada um com seu propósito e regras específicas. No primeiro momento abordaremos a questão principal no debate e o objetivo é que cada participante expresse sua opinião inicial sobre o tema central. Que foi aquele tema que lhes foi enviado. No segundo momento teremos uma rodada de perguntas direcionadas para cada um dos debatedores. Abriremos espaço para outros participantes contra argumentar, ou espressarem as suas opiniões sobre a resposta dada após as perguntas direcionadas, teremos uma última pergunta que será direcionada a todos os participantes, nesse momento, vocês têm a Liberdade de escolher se desejam responder ou não e só caso tenham algo no caso sintam que tem algo relevante a acrescentar. Por fim, no terceiro momento, será perguntado se os participantes têm alguma outra colocação final sobre o tema. Então cada um de vocês terá esse movimento inicial para expressar suas principais opiniões e pensamentos sobre o tema. E o tema é, esse que aí está e que foi também lhes enviado, então começando do debatedor número 1. Quais são as suas opiniões iniciais sobre o tema? Uma opinião inicial é que, conforme vai crescendo na inteligência artificial e se não for assim, não foi controlado o público que a utiliza, as pessoas vão acabar se dependendo, dependendo de mais dela, por causa de vai acabar usando ela para fazer qualquer coisa na vamos ver, o estudante vai usar a inteligência artificial para fazer as questões por de algum professor e vai acabar não se dedicando para aprender o assunto. Assim vai acabar sendo muito dependente dela e não vai acabar crescendo quase nada, na sociedade. Debatedor número 2. Eu acredito que o uso da da inteligência artificial de forma benigna, pode como o próprio já diz, causar um bem no sentido, por exemplo, aplicativos que utilizam da inteligência artificial, da IA, para produzir, por exemplo, na questão da construção civil, que pode utilizar para a questão de produção de planta baixa ou de outros projetos arquitetônicos nesse sentido olhando essa visão, eu acredito que é um uso com um bem explícito, mas que também depende, por exemplo, do seu uso como o debatedor 1 falou poderia no caso do estudante que ele começa a depender demais da da inteligência artificial, no nosso caso mais atual, por exemplo, o ChatGPT que para responder as perguntas de seja de exercício ou atividades, enfim, começa a depender o uso demasiado da inteligência artificial e pelo contrário não provocam bem a si mesmo. Ele está degenerando a si mesmo. Está seu tá agravando seu processo de de desconstrução de conhecimento, é isso que é isso que eu defendo, é a questão do uso mais moderado. Debatedor número 3. Eu acredito que a IA como qualquer outra tecnologia, ela vai ela vai ser útil para facilitar a nossa vida, mas como nas outras tecnologias, também tem que ser usado de uma da forma certa. Durante a guerra a gente viu que usou a tecnologia para fazer mal para a sociedade, a IA pode ser a mesma, pode ser usado para a mesma coisa e pode fazer um mal para você próprio. Então acho que ela deve ser usada, mas também tem que ser empregado no ponto de como usar ela da maneira certa, tem que ensinar as pessoas a usar da maneira correta. Debatedor número 4 Sinto que é necessário criar uma verificação no que está sendo passada para ela, uma vez que sem um limite eu posso infringir a própria ética no caso de criar, por exemplo, deep face, né? Fake news que é desenvolver né imagens né alteradas, então foi pelo. Acho que é bom é que exista uma evolução na na IA, mas deve ter assim, uma verificação, pra que que está sendo passado para ela não seja nada que infrinja a ética, a moral, como um todo. Debatedor número 5 Bem, eu acredito que esse, na verdade, é um assunto bastante delicado, né? É pessoas de diversas áreas, vão ter opiniões que divergem uma das outras É, mas como qualquer outra coisa, é o seu uso de forma imprópria é pode causar desses problemas, né? Então, é o uso de inteligência artificial, tem que ser definitivamente, tem que ser pensado antes de ser usado, né? Você não pode usar de uma forma indevida, ainda mais que ainda não está dentro de um de uma lei. Se, como você pode usar, como você não pode, então imagino que seja um assunto bem delicado. Pois bem, nós estaremos agora, então a rodada de perguntas, certo? A primeira rodada de perguntas. E nela eu irei realizar uma pergunta para cada um dos participantes, que terá seu tempo de resposta ao final da sua, ao final da sua resposta, onde os demais terão, é espaço para comentar algo sobre a pergunta ou a resposta dada. Então, vamos começar. A primeira pergunta para o levantador número 1. Se o sistema de IA generativo criar algo prejudicial ou ofensivo, por exemplo, uma imagem com conteúdo racista, quem deve ser responsabilizada? Desenvolvedor da IA, o usuário, a plataforma que hospeda com alguma outra instituição pessoa? Debatedor número 1. Acredito que seja um único, acredito que seja o usuário, que seja responsabilizado por causa que como já foi dito, pelo debatedor 3, número 3 a sobre a opinião dele, que a inteligência artificial se for utilizada de forma errada, acaba sendo mais a culpa da pessoa que está usando ela não de alguém que desenvolver, porque a pessoa que desenvolveu a internet artificial fez para com intuito de beneficiar a sociedade. Mas a pessoa que usou o usuário usou de forma errada e acaba fazendo alguma coisa prejudicial. Alguém tem algo a comentar sobre a resposta do debatedor um ou a pergunta em si. Tá, então passemos para a segunda pergunta para o debatedor número 2. IAs generativas podem ser usadas nos processos educacionais, por exemplo, em aulas, atividades ou provas, o ou a aluno ou a aluna deve reportar ao professor ou professora sobre o uso de IAs generativas em suas atividades? Eu acredito que pode ser utilizado sim, como qualquer outra tecnologia, a gente utiliza, por exemplo, em nossas atividades escolares acadêmicas. A questão do do uso da internet, por exemplo em sites de pesquisa,e a gente pode encontrar informações. Como a IA se baseia em um banco de dados gigante eu acredito que nós podemos utilizar no sentido de que podemos utilizá-la como fonte dessas informações, certo? Por exemplo, eu quero pesquisar, supondo, de uma forma bem grotesca sobre a questão da história da do Brasil. Eu Acredito que a IA vai nos fornecer é informações que serão necessárias ou muito importantes para o nosso crescimento, para o nosso crescimento intelectual de conhecimento e outra pergunta, o aluno deve reportar ao professor sobre o uso das IAs generativas em suas atividades? Eu lembro que se Eu Acredito que sim, como qualquer outro banco de dados que nós já vemos referir da onde a gente pegou essas informações. Eu acredito que é necessário também reportar ao professor, ao docente, o da onde a gente tirou essas informações. Alguém gostaria de comentar algo sobre a resposta dada ou a pergunta em si? Então passemos pra pergunta 3, ao debatedor número 3. De que maneira a propriedade intelectual deve ser tratada quando um conteúdo é gerado por uma IA? Por exemplo, se um usuário gerou uma música usando IA, o crédito pela criação deve ser o deste usuário? Da plataforma de IA utilizada? Do criador dos dados originais, com os quais a IA foi treinada? Ou de que?
 Não acredito que deva ser da IA porque a IA gerativa, ela é feita como uma base de dados muito gigantesca e ela vai fazer as coisas coisas novas, não vai replicar o que tem, ela vai aprender e vai replicar e vai criar coisas novas. Então, uma criação dela e uma criação de outras pessoas, ela, o crédito deve ser dela. Debatedor número 5 Eu imagino que seja, tenha créditos à inteligência artificial, mas que não totalmente, já que ela utiliza de um banco de dados, possivelmente ela vai estar pegando algo de outra pessoa e reutilizando, é de certa forma, então possivelmente, se você identifica é seus direitos autorais no que ela gerou, eu imagino que é o os direitos autorais você também tenha participação disso. Debatedor número 2. Como o debatedor falou, número 5, ele falou, é como se fosse uma árvore, né? Se a gente puxar, é de onde está havendo as informações sempre vai ter um crédito como ele falou, na questão do do do grande banco de dados, O o artistao músico que está procurando, é, criar uma nova música utilizando a inteligência artificial também deve ter seu crédito de certa forma, mas acredito também tanto os conteúdos é que estão dentro É da IA, do do banco de dados da IA também devem ser procurados de de onde foi que ela utilizou essas informações E, de certa forma, acredito que também até o os criadores do da IA também. Alguém mais, tem algo a comentar sobre a pergunta, sobre as respostas dadas? Não? Então sigamos para a pergunta 4 debatedor número 4. Como garantir que conteúdo gerados por IA não sejam usados para espelhar informações tendenciosas, errôneas ou maliciosas, por exemplo, quando esse tipo de informação é propagada massivamente por bots em uma rede social, a responsabilidade deve ser da empresa que administra a rede social, dos programadores da IA, de quem produziu aquele conteúdo usando a IA?   Eu prevejo que seja de quem produziu aquele conteúdo. Porque não minto minto que seja do da empresa o cara tem domínio né, daquela rede social de propagar aquilo que ela está passando. Então se passa uma informação que ela não consegue verificar, não consegue analisar que é justamente tendenciosa é errônea, né? Como uma imagem, né? Uma fake news, uma, um som, uma até uma, música que passe por assim sem ela perceber essa essa falha, né? Que pode dar na ética. Eu vejo que seja dela a essa essa falha, né não de que ade que a produziu, assim, quem produziu também, mas seria mais ela, que tá propagando. Debatedor 2 Eu Acredito também, assim como o debatedor 4, falou É, tem uma parcela em culpa da empresa em talvez propagar essa esse conteúdo de forma errônea ou de forma é replicada, mas também do próprio usuário que que produziu o conteúdo usando da IA. Acredito que a IA ela Não, não, não tem essa, tanto, essa parcela de culpa que que eu comparo com se fosse o carro, por exemplo, de forma grotesca, claro. Um carro, o criador de um automóvel, no caso ele não pensou naquelas pessoas, talvez muito provavelmente nas pessoas que poderiam utilizálo como 11 veículo de de de rachas na rua, por exemplo. Mas como uma forma de locomoção das pessoas e para um lugar, outros lugares et cetera, et cetera, mas que devido aos seus usuários, por exemplo, nós, quando nós utilizamos OOO nosso automóvel, assumimos essa essa responsabilidade de que, caso venha, venhamos a desrespeitar desrespeitar ou infringir leis de trânsitos, leis de trânsito perdão e assumirmos essas responsabilidades, assumimos essa culpa de talvez provocar um acidente e comparando de forma com o usuário que utiliza essa IA eu acredito que ele também deve ser responsabilizado. Por quê? Porque ele está assumindo de que quando ele produz esse conteúdo, é claro, é evidência evidencialmente vai ser 11 conteúdo é pejorativo, um conteúdo que é traz mal para a sociedade. Debatedor 5 Eu já para mim, eu não imagino que a empresa, a rede social que administra ela seria responsável, já que possivelmente vai haver algum algoritmo ele vai identificar aquilo, claro que existem métodos de filtragem para identificar se se é um, se é uma informação falsa ou não, mas é o algoritmo, só vai repassar para outras pessoas se as pessoas concordarem e curtirem compartilharem, repassarem para outras pessoas, então, eu imagino que é mais das pessoas que veem aquele conteúdo e de quem produziu esse conteúdo É a culpa de informações tendenciosas serem repassadas do que mesmo da responsabilidade da rede social, né? Já que, teoricamente, a rede social não foi feita para isso. Alguém tem algo a comentar sobre o que tinha dito ou sobre a pergunta? Então passemos a quinta pergunta ao debatedor número 5. Quais os impactos da IA no ambiente de trabalho, você acha que o uso de IAs generativas irá gerar ou destruir empregos, por exemplo, uma IA generativa que consegue fazer um trabalho de um arquiteto ou publicitário de forma mais rápida fará um trabalho totalmente confiável. É, eu imagino que E vai existir impactos dessa inteligência artificial, né no mercado de trabalho, mas isso acontece hoje em dia sempre aconteceu com diversas outras coisas existem diversos empregos que não existem mais porque foi criado algo novo, que é melhor, que é superado hoje em dia, por exemplo, não tem mais antigamente tinha um emprego nas naqueles lugares de jogar boliche, tinha um pessoal que organizava os pinos para você jogar de novo. Hoje em dia não é assim, mais não. Esse emprego não existe mais. Então, é enquanto a inteligência artificial estiver sendo é desenvolvida, é claro que você não pode confiar totalmente, né? É preciso que você tenha a é a inspeção de um arquiteto, de um cara que estudou para isso. Possivelmente a inteligência artificial vai ter arquitetos que vão ter, vão estar trabalhando em conjunto com programadores para desenvolver essa inteligência. Então, eu imagino que enquanto a inteligência artificial estiver evoluindo, a tendência é que ela esteja mais e mais confiável E diante disse empregos, se não forem atualizados, se não tiverem mais AA eficácia deixaram de existir. Alguém tem algo a comentar sobre esse ponto ou sobre a resposta dada? Bem, então passamos agora para a pergunta livre. A, lembrando que vocês podem responder ou não. Uso e o desenvolvimento de áreas narrativas devem ser fortemente fiscalizados por órgãos governamentais ou elas são apenas mais um tipo de software comum, como milhares de outros existentes? Debatedor número 5. É bastante adverso. Sim, elas devem ser fiscalizadas, mas não fortemente. Até porque tem que ter algum, alguma parte criativa naquilo, né? Se for totalmente fiscalizado, talvez você trave você pare o desenvolvimento daquilo, que você torne lento. O desenvolvimento disso, se você não corr Claro que a fiscalização é importante/> justamente por outros motivos, direitos autorais, tudo mais, mas é, se for fortemente, é fiscalizado, é A gente vai deixar de evoluir? Basicamente a gente vai deixar de ter um desenvolvimento mais rápido simplesmente por conta que o governo está fiscalizando e ele não deixa por conta N motivos, né? Então, imagino que deve ser fiscalizado, mas até certo ponto. Alguém mais gostaria de responder a pergunta ou comentar sobre o que foi dito? Bem, então agora cada participante terá um momento para falar suas considerações finais sobre o tema, sua opinião ou visão sobre o tema mudou depois do debate? Começando do debatedor número 1, você tem alguma coisa sobre para falar mais sobre o tema? Tem alguma concentração final, senão, você sua ou visão ou opinião mudou sobre o tema de IAs generativas depois do debate? Não. Debatedor 2 Eu acredito, só, para, a título de conclusão final, acredito que a inteligência artificial, assim como qualquer outra tecnologia, assim como na comparação que eu fiz como automóveis, de forma, grotesca, é claro, mas, ela pode auxiliar, ela vai auxiliar a sociedade na criação de, de, na produção de, de n coisas. No entanto, que deve ter assim um cuidado sim, uma fiscalização para que não torne essa, essa produção de forma demasiada ou que acabe por gerar, é, algo algo que não produza benefícios, pelo contrário, produza malefícios para a sociedade na questão, por exemplo, de criação de imagens utilizando fotos de pessoas e aí parece com que uma pessoa está em um canto que não está ou que está fazendo algo que não, nunca fez. Eu acho que isso acredito que isso deve ser considerado na fiscalização da, da IA para que não ocorra, por exemplo, se, se a IA, um usuário utiliza a minha imagem, usando também a IA e coloque a minha imagem como se eu tivesse fazendo algo ilícito. Não, claro que isso traz no benefício para mim, como pessoa, porque eu nunca fiz aquilo, porque eu nunca estive naquele lugar, et cetera, et cetera. Então deve ser fiscalizado sim, para que não ocorra esse tipo de produção. Debatedor 3 Não Debatedor 4 Prevejo que ela é uma ferramenta, uma conquista, na verdade, em toda espaço da, da, da, da computação, por exemplo, e que, sim, é, deve existir, né, uma fiscalização ali, não fortemente, é claro, mas para barrar justamente problemas, pois uma ferramenta como ela, é, ela é aberta ao público, é vai do usuário, né? O seu intuito, né? O que ele quer trazer, o que quer que a IA faça e retorne. Então, para não ter problemas como direitos autorais ou de problemas que infrinjam a ética, então é algo racista, né, algo homofóbico, por exemplo, uma imagem. Eu vejo que tem que ter um, um, limite, né? Um órgão que verifique, né, passe de parâmetro para a IA e o cara retorna. Debatedor 5. Só para finalizar, é, eu imagino que é um conteúdo, é um, é um, é um tema que, é, traz bastante medo na sociedade, né? Principalmente porque, a sociedade em geral, não conhece como a inteligência artificial funciona, o que que ela pode fazer, o que que ela não pode fazer. Então é preciso entender corretamente o que, que ela pode fazer, até onde ela vai chegar, o que é que ela pode te causar. E a partir disso, é a inteligência artificial vai ser mais aceita, né? Principalmente por gerações que já vem, né? E é isso. Então utilizem este QR Code para acessar e responder a um questionário, que é uma avaliação e avaliação geral do debate. Suas respostas são confidenciais e serão usadas para fins de avaliação e aprimoramento. Dito isto, obrigado pela participação de vocês no experimento, iremos fazermos um sorteio de um brinde no final de todos os experimentos. Fiquem atentos, portanto, ao e-mail que vocês cadastraram no formulário de inscrição.
"""

# Prompt 1.2 (Zero-shot some context)





In [ ]:
prompt = f"""
The following Portuguese text was generated using an STT (Speech To Text) model from a debate recording. Such transcriptions often contain unwanted elements known as disfluencies, which include:

- **Repetitions**: These occur when a word or phrase is repeated consecutively without adding any new meaning or value to the sentence. Repetitions are often present in speech but they do not contribute any additional information to the context. For example, in the phrase "o ambiente da da tecnologia," the word "da" is repeated twitce without adding anything meaningful to the sentence, only one occurence of "da" would be sufficient to get what the phrase is trying to transmit.

- **Hesitations**: These are non-verbal expressions or fillers used in speech to indicate a pause or hesitation. Common hesitations include sounds like "ahh," "ehh," "um,", "hmm" or "uh." They serve as verbal placeholders while the speaker thinks or searches for the right words. Although they can indicate thought processes, they do not add substantive meaning to the text and should be removed for clarity. For example, in the phrase "Eu estava, ahh, pensando sobre isso," the term "ahh" is a hesitation that can be removed.

- **Corrections**: These occur when a speaker makes an initial error in their speech and then corrects it. The correction typically involves an initial incorrect phrase followed by a revised version of the same phrase. For instance, in the phrase "que isso é, quer dizer, isso foi," the speaker initially says "que isso é," makes a correction by saying "quer dizer," and finally provides the corrected phrase "isso foi." The goal is to remove the error, leaving only the final corrected version.

Please remove these disfluencies from the text without altering the original meaning. You should NOT change text that does not refer to disfluencies. ANY text that is not in one of those three categories of disfluencies should not be changed. Return only the cleaned text with no additional information. Remember the text must be in its original full size, but without the disfluencies in it.


Text: {texto_disfluente}
"""

response2, duration2 = get_response_and_time(prompt)

print(response2)
print('A limpeza foi feita em ', duration2)

Vamos lá, boa tarde todo mundo. Bem-vindos a este debate. Promovido pelo grupo de pesquisa do lacina debate em educação. Eu me chamo David e gostaria de apresentar aqui quem está comigo, Klaywert que também faz parte do grupo. Mas aí a gente tem também Bryan e Hellen, que são tirando Hellen, a maioria estudante do mestrado, Hellen é estudante da graduação. Esse projeto está sobre orientação do professor Campelo, que é o professor que vocês conhecem. A razão pela qual nos reunimos aqui hoje, é para gerar uma base de dados de voz que posteriormente será transcrita e utilizada para avaliação e treinamento de modelos de inteligência artificial. Nosso foco está entre vários tópicos relacionados a debates, fazendo dessas iniciativa não apenas oportunidade de aprendizado e crescimento mas também uma contribuição valiosa para avanço de IA.

Vocês assinaram o termo, certo? Todo mundo já assinou o termo, então a gente pode pular essa parte. Cada debatedor vai receber uma identificação, a gente n

## prompt

In [ ]:
prompt2 = """Gravação iniciada. Minha identificação é o número aluno um, estou contribuindo para a pesquisa no Brasil. Minha identificação é aluno 2, estou contribuindo para a pesquisa no Brasil. Minha identificação é a número 3. Estou contribuindo com a pesquisa no Brasil. Minha eficação é o número 4. Estou contribuindo com a pesquisa do Brasil. Minha identificação número 5, estou contribuindo com a pesquisa no Brasil. Excelente. Pois bem, agora eu explicarei as regras deste debate, e então, antes de mergulharmos em nossa discussão, é essencial. Todos compreendam e sigam as reuniões de debate. Essas regras foram ampliadas para garantir um debate justo e ordenado para todos os envolvidos. A única principal regra é: sem interrupções. Os debatedores não devem interromper uns aos outros enquanto alguém estiver falando. Se você deseja contribuir para alguma discussão ou oferecer um contra argumento, por favor, levante a mão e aguarde um moderador, no caso eu lhe conceder a palavra, uma vez que tenha sido autorizada a palavra, você terá a palavra e poderá expressar seus pensamentos ou responder aos outros. Vamos seguir um formato estruturado dividido em 3 momentos distintos, cada um com seu propósito e regras específicas. No primeiro momento abordaremos a questão principal no debate e o objetivo é que cada participante expresse sua opinião inicial sobre o tema central. Que foi aquele tema que lhes foi enviado. No segundo momento teremos uma rodada de perguntas direcionadas para cada um dos debatedores. Abriremos espaço para outros participantes contra argumentar, ou espressarem as suas opiniões sobre a resposta dada após as perguntas direcionadas, teremos uma última pergunta que será direcionada a todos os participantes, nesse momento, vocês têm a Liberdade de escolher se desejam responder ou não, e só caso tenham algo no caso sintam que tem algo relevante a acrescentar. Por fim, no terceiro momento, será perguntado se os participantes têm alguma outra colocação final sobre o tema. Então cada um de vocês terá esse movimento inicial para expressar suas principais opiniões e pensamentos sobre o tema. E o tema é, esse que aí está e que foi também lhes enviado, então começando do debatedor número 1. Quais são as suas opiniões iniciais sobre o tema? Uma opinião inicial é que, conforme vai crescendo na inteligência artificial e se não for, não foi controlado o público que a utiliza, as pessoas vão acabar se dependendo, de mais dela, por causa de vai acabar usando ela para fazer qualquer coisa na, vamos ver, o estudante vai usar a inteligência artificial para fazer as questões por, de algum professor e vai acabar não se dedicando para aprender o assunto. Assim vai acabar sendo muito dependente dela e não vai acabar crescendo quase nada, na sociedade. Debatedor número 2. Eu acredito que o uso da inteligência artificial de forma benigna, pode como o próprio já diz, causar um bem no sentido, por exemplo, aplicativos que utilizam da inteligência artificial, da IA, para produzir, por exemplo, na questão da construção civil, que pode utilizar para a questão de produção de planta baixa ou de outros projetos arquitetônicos nesse sentido olhando essa visão, eu acredito que é um uso, com um bem explícito, mas que também depende, por exemplo, do seu uso como o debatedor 1 falou poderia no caso do estudante que ele começa a depender demais da inteligência artificial, no nosso caso mais atual, por exemplo, o ChatGPT que para responder as perguntas de seja de exercício ou atividades, enfim, começa a depender o uso demasiado da inteligência artificial e pelo contrário não provocam bem a si mesmo. Ele está degenerando a si mesmo. Está seu tá agravando seu processo de de desconstrução de conhecimento, é isso que é isso que eu defendo, é a questão do uso mais moderado. Debatedor número 3. Eu acredito que a IA como qualquer outra tecnologia, ela vai ela vai ser útil para facilitar a nossa vida, mas como nas outras tecnologias, também tem que ser usado de uma, da forma certa. Durante a guerra a gente viu que usou a tecnologia para fazer mal para a sociedade, a IA pode ser a mesma, pode ser usado para a mesma coisa e pode fazer um mal para você próprio. Então acho que ela deve ser usada, mas também tem que ser empregado no ponto de como usar ela da maneira certa, tem que ensinar as pessoas a usar da maneira correta. Debatedor número 4 Sinto que é necessário criar uma verificação no que está sendo passada para ela, uma vez que sem um limite eu posso infringir a própria ética no caso de criar, por exemplo, deep face, né? Fake news que é desenvolver né imagens né alteradas, então foi pelo. Acho que é bom é que exista uma evolução na na IA, mas deve ter assim, uma verificação, para que que está sendo passado para ela não seja nada que infrinja a ética, a moral, como um todo. Debatedor número 5 Bem, eu acredito que esse, na verdade, é um assunto bastante delicado, né? É pessoas de diversas áreas, vão ter opiniões que divergem uma das outras É, mas como qualquer outra coisa, é o seu uso de forma imprópria é pode causar desses problemas, né? Então, é o uso de inteligência artificial, tem que ser definitivamente, tem que ser pensado antes de ser usado, né? Você não pode usar de uma forma indevida, ainda mais que ainda não está dentro de um de uma lei. Se, como você pode usar, como você não pode, então imagino que seja um assunto bem delicado. Pois bem, nós estaremos agora, então a rodada de perguntas, certo? A primeira rodada de perguntas. E nela eu irei realizar uma pergunta para cada um dos participantes, que terá seu tempo de resposta ao final da sua, ao final da sua resposta, onde os demais terão, é espaço para comentar algo sobre a pergunta ou a resposta dada. Então, vamos começar. A primeira pergunta para o levantador número 1. Se o sistema de IA generativo criar algo prejudicial ou ofensivo, por exemplo, uma imagem com conteúdo racista, quem deve ser responsabilizada? Desenvolvedor da IA, o usuário, a plataforma que hospeda com alguma outra instituição pessoa? Debatedor número 1. Acredito que seja um único, acredito que seja o usuário, que seja responsabilizado por causa que como já foi dito, pelo debatedor 3,  número 3 a sobre a opinião dele, que a inteligência artificial se for utilizada de forma errada, acaba sendo mais a culpa da pessoa que está usando ela não de alguém que desenvolver, porque a pessoa que desenvolveu a internet artificial fez para com intuito de beneficiar a sociedade. Mas a pessoa que usou o usuário usou de forma errada e acaba fazendo alguma coisa prejudicial. Alguém tem algo a comentar sobre a resposta do debatedor um ou a pergunta em si. Tá, então passemos para a segunda pergunta para o debatedor número 2. IAs generativas podem ser usadas nos processos educacionais, por exemplo, em aulas, atividades ou provas, o ou a aluno ou a aluna deve reportar ao professor ou professora sobre o uso de IAs generativas em suas atividades? Eu acredito que pode ser utilizado sim, como qualquer outra tecnologia, a gente utiliza, por exemplo, em nossas atividades escolares acadêmicas. A questão do do uso da internet, por exemplo em sites de pesquisa,e a gente pode encontrar informações. Como a IA se baseia em um banco de dados gigante eu acredito que nós podemos utilizar no sentido de que podemos utilizá-la como fonte dessas informações, certo? Por exemplo, eu quero pesquisar, supondo, de uma forma bem grotesca sobre a questão da história da do Brasil. Eu Acredito que a IA vai nos fornecer é informações que serão necessárias ou muito importantes para o nosso crescimento, para o nosso crescimento intelectual de conhecimento e outra pergunta, o aluno deve reportar ao professor sobre o uso das IAs generativas em suas atividades? Eu lembro que se Eu Acredito que sim, como qualquer outro banco de dados que nós já vemos referir, da onde a gente pegou essas informações. Eu acredito que é necessário também reportar ao professor, ao docente, o da onde a gente tirou essas informações. Alguém gostaria de comentar algo sobre a resposta dada ou a pergunta em si? Então passemos pra pergunta 3, ao debatedor número 3. De que maneira a propriedade intelectual deve ser tratada quando um conteúdo é gerado por uma IA? Por exemplo, se um usuário gerou uma música usando IA, o crédito pela criação deve ser o deste usuário? Da plataforma de IA utilizada? Do criador dos dados originais, com os quais a IA foi treinada? Ou de que?
 Não acredito que deva ser da IA porque a IA gerativa, ela é feita como uma base de dados muito gigantesca e ela vai fazer as coisas coisas novas, não vai replicar o que tem, ela vai aprender e vai replicar e vai criar coisas novas. Então, uma criação dela e uma criação de outras pessoas, ela, o crédito deve ser dela. Debatedor número 5 Eu imagino que seja, tenha créditos à inteligência artificial, mas que não totalmente, já que ela utiliza de um banco de dados, possivelmente ela vai estar pegando algo de outra pessoa e reutilizando, é de certa forma, então possivelmente, se você identifica, é seus direitos autorais no que ela gerou, eu imagino que é o os direitos autorais você também tenha participação disso. Debatedor número 2. Como o debatedor falou, número 5, ele falou, é como se fosse uma árvore, né? Se a gente puxar, é de onde está havendo as informações sempre vai ter um crédito como ele falou, na questão do do do grande banco de dados, O o artistao músico que está procurando, é, criar uma nova música utilizando a inteligência artificial também deve ter seu crédito de certa forma, mas acredito também tanto os conteúdos é que estão dentro É da IA, do do banco de dados da IA também devem ser procurados de de onde foi que ela utilizou essas informações E, de certa forma, acredito que também até o os criadores do da IA também. Alguém mais, tem algo a comentar sobre a pergunta, sobre as respostas dadas? Não? Então sigamos para a pergunta 4 debatedor número 4. Como garantir que conteúdo gerados por IA não sejam usados para espelhar informações tendenciosas, errôneas ou maliciosas, por exemplo, quando esse tipo de informação é propagada massivamente por bots em uma rede social, a responsabilidade deve ser da empresa que administra a rede social, dos programadores da IA, de quem produziu aquele conteúdo usando a IA?   Eu prevejo que seja de quem produziu aquele conteúdo.  Porque não minto que seja do da empresa o cara tem domínio né, daquela rede social de propagar aquilo que ela está passando. Então se passa uma informação que ela não consegue verificar, não consegue analisar que é justamente tendenciosa é errônea, né? Como uma imagem, né? Uma fake news, uma, um som, uma até uma, música que passe por assim sem ela perceber essa essa falha, né? Que pode dar na ética. Eu vejo que seja dela a essa essa falha, né não de que ade que a produziu, assim, quem produziu também, mas seria mais ela, que tá propagando. Debatedor 2 Eu Acredito também, assim como o debatedor 4, falou É, tem uma parcela em culpa da empresa em talvez propagar essa esse conteúdo de forma errônea ou de forma é replicada, mas também do próprio usuário que que produziu o conteúdo usando da IA. Acredito que a IA ela  Não, não, não tem essa, tanto, essa parcela de culpa que que eu comparo com se fosse o carro, por exemplo, de forma grotesca, claro. Um carro, o criador de um automóvel, no caso ele não pensou naquelas pessoas, talvez muito provavelmente nas pessoas que poderiam utilizálo como veículo de de de rachas na rua, por exemplo. Mas como uma forma de locomoção das pessoas e  para um lugar, outros lugares et cetera, et cetera, mas que devido aos seus usuários, por exemplo, nós, quando nós utilizamos o nosso automóvel, assumimos essa essa responsabilidade de que, caso venha, venhamos a desrespeitar, leis de trânsito perdão e assumirmos essas responsabilidades, assumimos essa culpa de talvez provocar um acidente e comparando de forma com o usuário que utiliza essa IA eu acredito que ele também deve ser responsabilizado. Por quê? Porque ele está assumindo de que quando ele produz esse conteúdo, é claro, é evidência evidencialmente vai ser um conteúdo é pejorativo, um conteúdo que é traz mal para a sociedade. Debatedor 5 Eu já para mim, eu não imagino que a empresa, a rede social que administra ela seria responsável, já que possivelmente vai haver algum algoritmo ele vai identificar aquilo, claro que existem métodos de filtragem para identificar se se é um, se é uma informação falsa ou não, mas é o algoritmo, só vai repassar para outras pessoas se as pessoas concordarem e curtirem compartilharem, repassarem para outras pessoas, então, eu imagino que é mais das pessoas que veem aquele conteúdo e de quem produziu esse conteúdo É a culpa de informações tendenciosas serem repassadas do que mesmo da responsabilidade da rede social, né? Já que, teoricamente, a rede social não foi feita para isso. Alguém tem algo a comentar sobre o que tinha dito ou sobre a pergunta? Então passemos a quinta pergunta ao debatedor número 5. Quais os impactos da IA no ambiente de trabalho, você acha que o uso de IAs generativas irá gerar ou destruir empregos, por exemplo, uma IA generativa que consegue fazer um trabalho de um arquiteto ou publicitário de forma mais rápida fará um trabalho totalmente confiável. É, eu imagino que E vai existir impactos dessa inteligência artificial, né no mercado de trabalho, mas isso acontece hoje em dia sempre aconteceu com diversas outras coisas existem diversos empregos que não existem mais porque foi criado algo novo, que é melhor, que é superado hoje em dia, por exemplo, não tem mais antigamente tinha um emprego nas naqueles lugares de jogar boliche, tinha um pessoal que organizava os pinos para você jogar de novo. Hoje em dia não é assim, mais não. Esse emprego não existe mais. Então, é enquanto a inteligência artificial estiver sendo é desenvolvida, é claro que você não pode confiar totalmente, né? É preciso que você tenha a é a inspeção de um arquiteto, de um cara que estudou para isso. Possivelmente a inteligência artificial vai ter arquitetos que vão ter, vão estar trabalhando em conjunto com programadores para desenvolver essa inteligência. Então, eu imagino que enquanto a inteligência artificial estiver evoluindo, a tendência é que ela esteja mais e mais confiável E diante disse empregos, se não forem atualizados, se não tiverem mais a eficácia deixaram de existir. Alguém tem algo a comentar sobre esse ponto ou sobre a resposta dada? Bem, então passamos agora para a pergunta livre. A, lembrando que vocês podem responder ou não. Uso e o desenvolvimento de áreas narrativas devem ser fortemente fiscalizados por órgãos governamentais ou elas são apenas mais um tipo de software comum, como milhares de outros existentes? Debatedor número 5. É bastante adverso. Sim, elas devem ser fiscalizadas, mas não fortemente. Até porque tem que ter algum, alguma parte criativa naquilo, né? Se for totalmente fiscalizado, talvez você trave você pare o desenvolvimento daquilo, que você torne lento. O desenvolvimento disso, se você não, claro que a fiscalização é importante justamente por outros motivos, direitos autorais, tudo mais, mas é, se for fortemente, é fiscalizado, é A gente vai deixar de evoluir? Basicamente a gente vai deixar de ter um desenvolvimento mais rápido simplesmente por conta que o governo está fiscalizando e ele não deixa por conta N motivos, né? Então, imagino que deve ser fiscalizado, mas até certo ponto. Alguém mais gostaria de responder a pergunta ou comentar sobre o que foi dito? Bem, então agora cada participante terá um momento para falar suas considerações finais sobre o tema, sua opinião ou visão sobre o tema mudou depois do debate? Começando do debatedor número 1, você tem alguma coisa sobre para falar mais sobre o tema? Tem alguma concentração final, senão, você sua ou visão ou opinião mudou sobre o tema de IAs generativas depois do debate? Não. Debatedor 2 Eu acredito, só, para, a título de conclusão final, acredito que a inteligência artificial, assim como qualquer outra tecnologia, assim como na comparação que eu fiz como automóveis, de forma, grotesca, é claro, mas, ela pode auxiliar, ela vai auxiliar a sociedade na criação de, de, na produção de, de n coisas. No entanto, que deve ter assim um cuidado sim, uma fiscalização para que não torne essa, essa produção de forma demasiada ou que acabe por gerar, é, algo algo que não produza benefícios, pelo contrário, produza malefícios para a sociedade na questão, por exemplo, de criação de imagens utilizando fotos de pessoas e aí parece com que uma pessoa está em um canto que não está ou que está fazendo algo que não, nunca fez. Eu acho que isso acredito que isso deve ser considerado na fiscalização da, da IA para que não ocorra, por exemplo, se, se a IA, um usuário utiliza a minha imagem, usando também a IA e coloque a minha imagem como se eu tivesse fazendo algo ilícito. Não, claro que isso traz no benefício para mim, como pessoa, porque eu nunca fiz aquilo, porque eu nunca estive naquele lugar, et cetera, et cetera. Então deve ser fiscalizado sim, para que não ocorra esse tipo de produção. Debatedor 3 Não Debatedor 4 Prevejo que ela é uma ferramenta, uma conquista, na verdade, em toda espaço da, da, da, da computação, por exemplo, e que, sim, é, deve existir, né, uma fiscalização ali, não fortemente, é claro, mas para barrar justamente problemas, pois uma ferramenta como ela, é, ela é aberta ao público, é vai do usuário, né? O seu intuito, né? O que ele quer trazer, o que quer que a IA faça e retorne. Então, para não ter problemas como direitos autorais ou de problemas que infrinjam a ética, então é algo racista, né, algo homofóbico, por exemplo, uma imagem. Eu vejo que tem que ter um, um, um, limite, né? Um órgão que verifique, né, passe de parâmetro para a IA e o cara retorna. Debatedor 5. Só para finalizar, é, eu imagino que é um conteúdo, é um, é um, é um tema que, é, traz bastante medo na sociedade, né? Principalmente porque, a sociedade em geral, não conhece como a inteligência artificial funciona, o que que ela pode fazer, o que que ela não pode fazer. Então é preciso entender corretamente o que, que ela pode fazer, até onde ela vai chegar, o que é que ela pode te causar. E a partir disso, é a inteligência artificial vai ser mais aceita, né? Principalmente por gerações que já vem, né? E é isso. Então utilizem este QR Code para acessar e responder a um questionário, que é uma avaliação e avaliação geral do debate. Suas respostas são confidenciais e serão usadas para fins de avaliação e aprimoramento. Dito isto, obrigado pela participação de vocês no experimento, iremos fazermos um sorteio de um brinde no final de todos os experimentos. Fiquem atentos, portanto, ao e-mail que vocês cadastraram no formulário de inscrição.
"""

# Prompt 2.1 (Few shot some context)

In [ ]:
prompt = f"""
Context: The text below is a transcription in Portuguese from a Speech-to-Text (STT) model of a debate. In these transcriptions, disfluencies such as hesitations, repetitions, and corrections are common.

- **Repetitions**: These occur when a word or phrase is repeated consecutively without adding any new meaning or value to the sentence. Repetitions are often present in speech but they do not contribute any additional information to the context. For example, in the phrase "o ambiente da da tecnologia," the word "da" is repeated twitce without adding anything meaningful to the sentence, only one occurence of "da" would be sufficient to get what the phrase is trying to transmit.

- **Hesitations**: These are non-verbal expressions or fillers used in speech to indicate a pause or hesitation. Common hesitations include sounds like "ahh," "ehh," "um,", "hmm" or "uh." They serve as verbal placeholders while the speaker thinks or searches for the right words. Although they can indicate thought processes, they do not add substantive meaning to the text and should be removed for clarity. For example, in the phrase "Eu estava, ahh, pensando sobre isso," the term "ahh" is a hesitation that can be removed.

- **Corrections**: These occur when a speaker makes an initial error in their speech and then corrects it. The correction typically involves an initial incorrect phrase followed by a revised version of the same phrase. For instance, in the phrase "que isso é, quer dizer, isso foi," the speaker initially says "que isso é," makes a correction by saying "quer dizer," and finally provides the corrected phrase "isso foi." The goal is to remove the error, leaving only the final corrected version.

 The example below shows the same transcription in three stages. The first stage, "Original," is the raw transcription as it comes from the STT model. The second stage, "Marked," is the same transcription with tags identifying the disfluencies in the text, such as "<hes" for hesitations, "<erro" for errors, "<corr" for corrections that come after an error, and "<rep" for repeated words. The third and final stage shows the text without the marked transcriptions, cleaned of these so-called disfluencies.

Example:

"Original": "Excelente. Oi, bem, agora eu eu IA explicar as regras deste debate, e então, antes de de mergulharmos em nossa discussão, é essencial. Todos compreendam e sigam as reuniões de debate. Essas regras foram ampliadas para garantir um debate justo e ordenado para todos os envolvidos. A única principal regra é essa, sem interrupções. Os debatedores não devem interromper uns aos outros enquanto alguém estiver falando. Se você deseja contribuir para contra discussão ou oferecer um contra argumento, por favor, levando-se a mão e aguarde em moderador caso eu lhe conceder a palavra, uma vez que tenha sido autorizada a palavra, você terá a palavra e poderá ingressar seus pensamentos. Ou responder aos outros? Ah, vamos seguir um formato estruturado? Deveria em 3 momentos distintos, cada um com seu propósito e regras específicas. O primeiro momento abordaremos a questão principal no debate e o objetivo é que cada participante expresse sua opinião inicial sobre o tema central foi a medida que isso foi *****. O segundo momento teremos uma rodada de bernanke direcionadas para cada um dos debatedores. Abriremos espaço para outros participantes contra argumentar, eu experçarem as suas opiniões sobre a resposta dada após as Pendências direcionadas, teremos uma última pergunta que será direcionada a todos os participantes, nesse momento, vocês têm a Liberdade de escolher se desejam responder ou não? Ah, e só caso tenham muito algo no caso simples, que tem algo relevante a acrescentar. Por fim, no no terceiro momento, será perguntado se os participantes têm alguma outra colocação ao sobre o tema então. Cada um de vocês terá esse movimento inicial para lhes pensar suas principais opiniões e pensamentos sobre o terminal e o tema é, disse que aí está e que foi também seguido então, começando do debatedor número 1. Quais são as suas opiniões iniciais sobre o tema?"

"Marked": "Excelente. Pois bem, agora <rep eu eu/> explicarei as regras deste debate, e então, antes <rep de de/> mergulharmos em nossa discussão, é essencial. Todos compreendam e sigam as reuniões de debate. Essas regras foram ampliadas para garantir um debate justo e ordenado para todos os envolvidos. A única principal regra é: sem interrupções. Os debatedores não devem interromper uns aos outros enquanto alguém estiver falando. Se você deseja contribuir para alguma discussão ou oferecer um contra argumento, por favor, levante a mão e aguarde um moderador, no caso eu lhe conceder a palavra, uma vez que tenha sido autorizada a palavra, você terá a palavra e poderá expressar seus pensamentos ou responder aos outros. <hes ahh/>, vamos seguir um formato estruturado dividido em 3 momentos distintos, cada um com seu propósito e regras específicas. No primeiro momento abordaremos a questão principal no debate e o objetivo é que cada participante expresse sua opinião inicial sobre o tema central. Que foi aquele tema que lhes foi enviado. No segundo momento teremos uma rodada de perguntas direcionadas para cada um dos debatedores. Abriremos espaço para outros participantes contra argumentar, ou expressarem as suas opiniões sobre a resposta dada após as perguntas direcionadas, teremos uma última pergunta que será direcionada a todos os participantes, nesse momento, vocês têm a Liberdade de escolher se desejam responder ou não <hes ahh/>, <erro e só caso tenham algo /> <corr no caso sintam que tem algo/> relevante a acrescentar. Por fim, <rep no no/> terceiro momento, será perguntado se os participantes têm alguma outra colocação final sobre o tema. Então cada um de vocês terá esse movimento inicial para expressar suas principais opiniões e pensamentos sobre o tema. E o tema é, esse que aí está e que foi também lhes enviado, então começando do debatedor número 1. Quais são as suas opiniões iniciais sobre o tema?"

"Cleaned": "Excelente. Pois bem, agora eu explicarei as regras deste debate, e então, antes de mergulharmos em nossa discussão, é essencial. Todos compreendam e sigam as reuniões de debate. Essas regras foram ampliadas para garantir um debate justo e ordenado para todos os envolvidos. A única principal regra é: sem interrupções. Os debatedores não devem interromper uns aos outros enquanto alguém estiver falando. Se você deseja contribuir para alguma discussão ou oferecer um contra-argumento, por favor, levante a mão e aguarde um moderador, no caso de lhe concederem a palavra. Uma vez que tenha sido autorizado a falar, você terá a palavra e poderá expressar seus pensamentos ou responder aos outros. Vamos seguir um formato estruturado dividido em 3 momentos distintos, cada um com seu propósito e regras específicas. No primeiro momento, abordaremos a questão principal no debate e o objetivo é que cada participante expresse sua opinião inicial sobre o tema central. No segundo momento, teremos uma rodada de perguntas direcionadas a cada um dos debatedores. Abriremos espaço para outros participantes contra-argumentarem ou expressarem suas opiniões sobre as respostas dadas. Após as perguntas direcionadas, teremos uma última pergunta que será direcionada a todos os participantes. Nesse momento, vocês têm a liberdade de escolher se desejam responder ou não. Caso sintam que têm algo relevante a acrescentar. Por fim, no terceiro momento, será perguntado se os participantes têm alguma outra colocação final sobre o tema. Então, cada um de vocês terá esse momento inicial para expressar suas principais opiniões e pensamentos sobre o tema. Começando pelo debatedor número 1, quais são suas opiniões iniciais sobre o tema?"

Following the example presented to you, remove all disfluencies from the following text without altering the original meaning or structure. You should NOT change text that does not refer to disfluencies. ANY text that is not in one of those three categories of disfluencies should not be changed. Return only the cleaned text with no additional information. Remember the text must be in its original full size, but without the disfluencies in it.

Text: {texto_disfluente}
"""

response3, duration3 = get_response_and_time(prompt)

print(response3)
print('A limpeza foi feita em ', duration3)

Vamos lá, é boa tarde todo mundo. Bem-vindos a este debate promovido pelo grupo de pesquisa do Lacina Debate em Educação. Eu me chamo David, e gostaria de apresentar aqui quem está comigo, Klaywert, que também faz parte do grupo. Mas temos também Bryan e Hellen, que são, tirando Hellen, a maioria estudante do mestrado; Hellen é estudante da graduação. Esse projeto está sob orientação do professor Campelo, que é o professor que vocês conhecem e que vocês viram agora. A razão pela qual nos reunimos aqui hoje é gerar uma base de dados de voz que posteriormente será transcrita e usada para avaliação e treinamento de modelos de inteligência artificial. Nosso foco está entre vários tópicos relacionados a debates, fazendo desta iniciativa não apenas uma oportunidade de aprendizado e crescimento, mas também uma contribuição valiosa para o avanço da IA.

Vocês assinaram o termo, certo? Todo mundo já assinou o termo; então, podemos pular essa parte. Cada debatedor vai receber uma identificação, 

# Chain-of-Thought Prompt

In [ ]:
prompt = f"""
Context: The text below is a transcription in Portuguese from a Speech-to-Text (STT) model of a debate. In these transcriptions, disfluencies such as hesitations, repetitions, and corrections are common.

- **Repetitions**: These occur when a word or phrase is repeated consecutively without adding any new meaning or value to the sentence. Repetitions are often present in speech but they do not contribute any additional information to the context. For example, in the phrase "o ambiente da da tecnologia," the word "da" is repeated twitce without adding anything meaningful to the sentence, only one occurence of "da" would be sufficient to get what the phrase is trying to transmit.

- **Hesitations**: These are non-verbal expressions or fillers used in speech to indicate a pause or hesitation. Common hesitations include sounds like "ahh," "ehh," "um,", "hmm" or "uh." They serve as verbal placeholders while the speaker thinks or searches for the right words. Although they can indicate thought processes, they do not add substantive meaning to the text and should be removed for clarity. For example, in the phrase "Eu estava, ahh, pensando sobre isso," the term "ahh" is a hesitation that can be removed.

- **Corrections**: These occur when a speaker makes an initial error in their speech and then corrects it. The correction typically involves an initial incorrect phrase followed by a revised version of the same phrase. For instance, in the phrase "que isso é, quer dizer, isso foi," the speaker initially says "que isso é," makes a correction by saying "quer dizer," and finally provides the corrected phrase "isso foi." The goal is to remove the error, leaving only the final corrected version.

Your task is to identify and remove these disfluencies while maintaining the original meaning and structure of the text. We will do this step by step, identifying and removing each type of disfluency.

Example 1:

Step 1: Identify Repetitions

Original: "Oi, eu eu acho que que é importante começar."
Marked: "Oi, <rep eu eu/> acho que <rep que que/> é importante começar."
Action: Remove repeated words.
Cleaned: "Oi, eu acho que é importante começar."

Step 2: Identify Hesitations

Original: "Ah, bem, então..."
Marked: "<hes Ah,/> bem, então..."
Action: Remove hesitation fillers like "Ah," "é," etc.

Step 3: Identify Corrections

Original: "Esse é, quero dizer, isso é essencial."
Marked: "<erro Esse é, quero dizer,/> <corr isso é essencial./>"
Action: Remove filler words during corrections.
Cleaned: "isso é essencial."

Example 2:

Step 1: Identify Repetitions

Original: "Bem, a gente tem que, ah, analisar mais profundamente."
Marked: "Bem, a gente tem que, <rep analisar analisar/> mais profundamente."
Action: No repetition to remove.
Cleaned: "Bem, a gente tem que, analisar mais profundamente."


Step 2: Identify Hesitations

Original: "Ah, analisar mais profundamente."
Marked: "<hes Ah,/> analisar mais profundamente."
Action: Remove hesitation "Ah."
Cleaned: "analisar mais profundamente."


Step 3: Identify Corrections

Original: "e só caso tenham algo, no caso sintam que tem algo"
Marked: " <erro e só caso tenham algo /> <corr no caso sintam que tem algo/>"
Action: Remove errors and keep the corrections".
Marked: "no caso sintam que tem algo"

Instructions: Using the step-by-step method demonstrated in the examples, identify and remove all disfluencies from the following text. Ensure to maintain the original transcription and structure. You should NOT change text that does not refer to disfluencies. ANY text that is not in one of those three categories of disfluencies should not be changed. Return only the cleaned text with no additional information. Remember the text must be in its original full size, but without the disfluencies in it.

Text: {texto_disfluente}
"""

response4, duration4 = get_response_and_time(prompt)

print(response4)
print('A limpeza foi feita em ', duration4)

Texto Limpo:

Vamos lá, boa tarde todo mundo. Bem-vindos a este debate. Promovido pelo grupo de pesquisa do lacina debate em educação. Eu me chamo David e gostaria de apresentar aqui, quem está comigo, Klaywert que também faz parte do grupo. Mas aí a gente tem também Bryan e Hellen, que são tirando Hellen, a maioria estudante do mestrado, Hellen é estudante da graduação. Esse projeto está sobre orientação do professor Campelo, que é o professor que vocês conhecem. A razão pela qual nos reunimos aqui hoje, é gerar uma base de dados de voz que posteriormente será transcrita e utilizada para avaliação e treinamento de modelos de inteligência artificial. Nosso foco está entre vários tópicos relacionados a debates, fazendo dessas iniciativa não apenas oportunidade de aprendizado e crescimento mas também uma contribuição valiosa para avanço de IA.

Vocês assinaram o termo, né? Todo mundo já assinou o termo, então a gente pode pular essa parte. Cada debatedor vai receber uma identificação, né

# Verificando a quantidade de disfluências removidas por cada prompt em comparação com o texto original

In [ ]:
hes_original, rep_original, erro_original, corr_original = count_disfluencias_in_clean(texto_limpo, hes, rep, erro, corr)

# Texto limpo por LLM Prompt 1.1 (Zero-shot no context)

## Resultado

In [ ]:

levenshtein_data = calculate_metrics(texto_limpo, response1)
print(levenshtein_data)
print()
hes_llm, rep_llm, erro_llm, corr_llm = count_disfluencias_in_clean(response1, hes, rep, erro, corr)

matches_repetições, length_of_incorrect_matches_repeticoes = count_matches(rep_original, rep_llm)
matches_hesitações, length_of_incorrect_matches_hesitações = count_matches(hes_original, hes_llm)
matches_erros, length_of_incorrect_matches_erros = count_matches(erro_original, erro_llm)
matches_correções, length_of_incorrect_matches_correções = count_matches(corr_original, corr_llm)

resultados = {
    'Repetições': (matches_repetições, len(rep_original)),
    'Hesitações': (matches_hesitações, len(hes_original)),
    'Erros': (matches_erros, len(erro_original)),
    'Correções': (matches_correções, len(corr_original))
}

total_length_of_incorrect_matches = (length_of_incorrect_matches_repeticoes + length_of_incorrect_matches_hesitações + length_of_incorrect_matches_erros + length_of_incorrect_matches_correções)
print(adjust_levenshtein_distance(levenshtein_data, total_length_of_incorrect_matches))
print(total_length_of_incorrect_matches)
print(exibir_metricas(resultados))

{'Levenshtein Distance': '19,768', 'Levenshtein Similarity': '46.69%', 'Edit Distance': '19,768', 'Original Length': '28,394', 'Processed Length': '9,707', 'Original Word Count': '5,022', 'Processed Word Count': '1,629'}

{'Adjusted Levenshtein Distance': 17925, 'Adjusted Levenshtein Similarity': '-84.66%'}
1843
Repetições:
  Acertos da LLM: 35
  Total de chaves no manual: 39
  Porcentagem de acertos: 89.74%
  Número de chaves não corretamente identificadas: 4

Hesitações:
  Acertos da LLM: 6
  Total de chaves no manual: 8
  Porcentagem de acertos: 75.00%
  Número de chaves não corretamente identificadas: 2

Erros:
  Acertos da LLM: 8
  Total de chaves no manual: 25
  Porcentagem de acertos: 32.00%
  Número de chaves não corretamente identificadas: 17

Correções:
  Acertos da LLM: 3
  Total de chaves no manual: 26
  Porcentagem de acertos: 11.54%
  Número de chaves não corretamente identificadas: 23

Média de porcentagem de acertos entre Hesitações, Erros e Repetições: 68.06%
None


# Texto limpo por LLM Prompt 1.2 (Zero-shot some context)

## Texto

## Resultado

In [ ]:

levenshtein_data = calculate_metrics(texto_limpo, response2)
print(levenshtein_data)
print()
hes_llm, rep_llm, erro_llm, corr_llm = count_disfluencias_in_clean(response2, hes, rep, erro, corr)

matches_repetições, length_of_incorrect_matches_repeticoes = count_matches(rep_original, rep_llm)
matches_hesitações, length_of_incorrect_matches_hesitações = count_matches(hes_original, hes_llm)
matches_erros, length_of_incorrect_matches_erros = count_matches(erro_original, erro_llm)
matches_correções, length_of_incorrect_matches_correções = count_matches(corr_original, corr_llm)

resultados = {
    'Repetições': (matches_repetições, len(rep_original)),
    'Hesitações': (matches_hesitações, len(hes_original)),
    'Erros': (matches_erros, len(erro_original)),
    'Correções': (matches_correções, len(corr_original))
}

total_length_of_incorrect_matches = (length_of_incorrect_matches_repeticoes + length_of_incorrect_matches_hesitações + length_of_incorrect_matches_erros + length_of_incorrect_matches_correções)
print(adjust_levenshtein_distance(levenshtein_data, total_length_of_incorrect_matches))
print(total_length_of_incorrect_matches)
print(exibir_metricas(resultados))

{'Levenshtein Distance': '22,196', 'Levenshtein Similarity': '35.77%', 'Edit Distance': '22,196', 'Original Length': '28,394', 'Processed Length': '6,432', 'Original Word Count': '5,022', 'Processed Word Count': '1,064'}

{'Adjusted Levenshtein Distance': 20080, 'Adjusted Levenshtein Similarity': '-212.19%'}
2116
Repetições:
  Acertos da LLM: 35
  Total de chaves no manual: 39
  Porcentagem de acertos: 89.74%
  Número de chaves não corretamente identificadas: 4

Hesitações:
  Acertos da LLM: 6
  Total de chaves no manual: 8
  Porcentagem de acertos: 75.00%
  Número de chaves não corretamente identificadas: 2

Erros:
  Acertos da LLM: 7
  Total de chaves no manual: 25
  Porcentagem de acertos: 28.00%
  Número de chaves não corretamente identificadas: 18

Correções:
  Acertos da LLM: 3
  Total de chaves no manual: 26
  Porcentagem de acertos: 11.54%
  Número de chaves não corretamente identificadas: 23

Média de porcentagem de acertos entre Hesitações, Erros e Repetições: 66.67%
None


# Texto limpo por LLM Prompt 2.1 (Few shot some context)

## Resultado

In [ ]:

levenshtein_data = calculate_metrics(texto_limpo, response3)
print(levenshtein_data)
print()
hes_llm, rep_llm, erro_llm, corr_llm = count_disfluencias_in_clean(response3, hes, rep, erro, corr)

matches_repetições, length_of_incorrect_matches_repeticoes = count_matches(rep_original, rep_llm)
matches_hesitações, length_of_incorrect_matches_hesitações = count_matches(hes_original, hes_llm)
matches_erros, length_of_incorrect_matches_erros = count_matches(erro_original, erro_llm)
matches_correções, length_of_incorrect_matches_correções = count_matches(corr_original, corr_llm)

resultados = {
    'Repetições': (matches_repetições, len(rep_original)),
    'Hesitações': (matches_hesitações, len(hes_original)),
    'Erros': (matches_erros, len(erro_original)),
    'Correções': (matches_correções, len(corr_original))
}

total_length_of_incorrect_matches = (length_of_incorrect_matches_repeticoes + length_of_incorrect_matches_hesitações + length_of_incorrect_matches_erros + length_of_incorrect_matches_correções)
print(adjust_levenshtein_distance(levenshtein_data, total_length_of_incorrect_matches))
print(total_length_of_incorrect_matches)
print(exibir_metricas(resultados))

{'Levenshtein Distance': '18,258', 'Levenshtein Similarity': '53.12%', 'Edit Distance': '18,258', 'Original Length': '28,394', 'Processed Length': '11,557', 'Original Word Count': '5,022', 'Processed Word Count': '1,934'}

{'Adjusted Levenshtein Distance': 16553, 'Adjusted Levenshtein Similarity': '-43.23%'}
1705
Repetições:
  Acertos da LLM: 34
  Total de chaves no manual: 39
  Porcentagem de acertos: 87.18%
  Número de chaves não corretamente identificadas: 5

Hesitações:
  Acertos da LLM: 6
  Total de chaves no manual: 8
  Porcentagem de acertos: 75.00%
  Número de chaves não corretamente identificadas: 2

Erros:
  Acertos da LLM: 9
  Total de chaves no manual: 25
  Porcentagem de acertos: 36.00%
  Número de chaves não corretamente identificadas: 16

Correções:
  Acertos da LLM: 4
  Total de chaves no manual: 26
  Porcentagem de acertos: 15.38%
  Número de chaves não corretamente identificadas: 22

Média de porcentagem de acertos entre Hesitações, Erros e Repetições: 68.06%
None


# Texto limpo por LLM Prompt COT

## Resultado

In [ ]:

levenshtein_data = calculate_metrics(texto_limpo, response4)
print(levenshtein_data)
print()
hes_llm, rep_llm, erro_llm, corr_llm = count_disfluencias_in_clean(response4, hes, rep, erro, corr)

matches_repetições, length_of_incorrect_matches_repeticoes = count_matches(rep_original, rep_llm)
matches_hesitações, length_of_incorrect_matches_hesitações = count_matches(hes_original, hes_llm)
matches_erros, length_of_incorrect_matches_erros = count_matches(erro_original, erro_llm)
matches_correções, length_of_incorrect_matches_correções = count_matches(corr_original, corr_llm)

resultados = {
    'Repetições': (matches_repetições, len(rep_original)),
    'Hesitações': (matches_hesitações, len(hes_original)),
    'Erros': (matches_erros, len(erro_original)),
    'Correções': (matches_correções, len(corr_original))
}

total_length_of_incorrect_matches = (length_of_incorrect_matches_repeticoes + length_of_incorrect_matches_hesitações + length_of_incorrect_matches_erros + length_of_incorrect_matches_correções)
print(adjust_levenshtein_distance(levenshtein_data, total_length_of_incorrect_matches))
print(total_length_of_incorrect_matches)
print(exibir_metricas(resultados))

{'Levenshtein Distance': '23,289', 'Levenshtein Similarity': '30.56%', 'Edit Distance': '23,289', 'Original Length': '28,394', 'Processed Length': '5,355', 'Original Word Count': '5,022', 'Processed Word Count': '882'}

{'Adjusted Levenshtein Distance': 21116, 'Adjusted Levenshtein Similarity': '-294.32%'}
2173
Repetições:
  Acertos da LLM: 35
  Total de chaves no manual: 39
  Porcentagem de acertos: 89.74%
  Número de chaves não corretamente identificadas: 4

Hesitações:
  Acertos da LLM: 6
  Total de chaves no manual: 8
  Porcentagem de acertos: 75.00%
  Número de chaves não corretamente identificadas: 2

Erros:
  Acertos da LLM: 7
  Total de chaves no manual: 25
  Porcentagem de acertos: 28.00%
  Número de chaves não corretamente identificadas: 18

Correções:
  Acertos da LLM: 3
  Total de chaves no manual: 26
  Porcentagem de acertos: 11.54%
  Número de chaves não corretamente identificadas: 23

Média de porcentagem de acertos entre Hesitações, Erros e Repetições: 66.67%
None
